<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 4: Distillation and Retrieval Augmented Generation</h2>
<h4>Lecture: Dr. Guokan Shang<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, November 18, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>Novemver 23
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

# Install Requirements

In [ ]:
# Install required dependencies
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate vllm
!pip install jedi
# !pip -q install datasets==2.21.0 pandas==2.2.2
# !pip -q install chromadb==0.4.22
# !pip -q install "numpy<2.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/

# Part 1 - Model Distillation

> In this section, you’ll learn the difference between **white-box** and **black-box** distillation, generate **synthetic data** to train a **student model**, and implement **white-box distillation** to specialize a model for a **RAG on Wikipedia** use case.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:</br>
Explain briefly the difference between black-box and white-box distillation? </br> What are the advantages and inconvenients of each approach?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Your answer here.

Black-box distillation is when the student model learns from the outputs of the teacher model without access to the teacher's internal parameters, hidden states, logits, or architecture.

The main advantages are that it works even when the teacher is closed-source, and it's easy to use, meaning that we would only need to use sample outputs and train the student on them.

The main inconvenients include a less efficiency, since you get less information per example. Another inconvenient is that we require many more queries and much more data.

White-box distillation is when the student model learns using internal information from the teacher model, including hidden states, attention maps, and all the components mentioned earlier.

The main advantages are that it's more efficient than the black box distillation, requiring fewer samples for a better performance. It can also distill architecture-secific knowledge, such as the attention patterns.

The main inconvenients are that it's only possible when teacher is open-source, and it is computationally more complex.
<hr style="border:10px solid green"> </hr>
</font></h4>



<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 2:</br>
What is the main requirement for a teacher/student pair of models to perform white-box distillation?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
Your answer here.

The main requirement for white box distillation for white-box distillation is that the teacher and student models must have compatible architectures that allow for alignment of their internal representations. In general, this means they should have similar layer structures, hidden dimensions, or attention mechanisms to enable meaningful transfer of intermediate knowledge.
<hr style="border:10px solid green"> </hr>
</font></h4>




## 1.1 - Synthetic Data Generation

We're going to specialize a small 0.5B parameter model to perform RAG by distilling the abilities of a 7B parameter one.

For that we'll be using `Qwen/Qwen2.5-0.5B-Instruct` as student and `Qwen/Qwen2.5-7B-Instruct-AWQ` (quantized version of `Qwen2.5-7B-Instruct`) as teacher.  

In order to perform white-box distillation on generated answers we have two choices.

1. We can perform a forward pass with the teacher, a forward pass with student on the complete sequence, and backprop difference of logprobs using KL Loss.
2. Generation of samples with the teacher, save the logprobs and perform finetuning in a second step.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 3:</br>
What are the computational advantages of 1. vs 2.?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>
Your answer here.

- In the option 1, we have more memory efficiency since we don't need to store large amounts of intermediate data such as the teacher logprobs as there won't be second pass over data, although it requires both models to be loaded simultaneously.
- In the option 2, it allows a better parallelization and can be more stable since the student training becomes very cheap due to no teacher forward, but requires significant storage for the generated samples.


<hr style="border:10px solid green"> </hr>
</font>

</h4>



We're going to generate a bunch of questions related to wikipedia paragraphs.
For that we need to establish a system prompt that will allow for easy extraction.

In [ ]:
system_prompt = """
You are a question generator.
The user will provide:

```json
{"title": "the title of an article", "paragraph": "a paragraph from that article"}
```

Your task:

* Generate one clear, self-contained question that can be answered using only the provided paragraph.
* The question must be **specific**, **unambiguous**, and directly tied to the paragraph’s content.
* Return the result with the question as a valid JSON** in the form:

```json
{
  "question": "your question here"
}
```

Example:
User input:

```json
{
"title": "The Moon Landing",
"paragraph": "On July 20, 1969, Neil Armstrong became the first human to set foot on the Moon, followed by Buzz Aldrin."
}
```
Assistant output:

```json
{
  "question": "Who was the first human to set foot on the Moon during the Apollo 11 mission?"
}
```
"""

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 4:</br>
In this system prompt, we don't generate answers, only questions. Explain why it's necessary in the context of white-box distillation.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
Your answer here.

In white-box distillation, we only generate questions because we want the student model to learn the teacher's reasoning process and answer generation patterns directly. By having the teacher generate both questions and answers during training, the student can learn the complete generation distribution rather than just learning to match pre-generated answers.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 5:</br>
For synthetic data generation we'll be using vLLM.
vLLM is an optimized llm inference engine that can improve generation speed thanks to hardware specific optimization and computational tricks such as Prefix KV Caching.
(https://docs.vllm.ai/en/latest/features/automatic_prefix_caching.html)</br></br> 1. Explain why prefix caching will be very efficient in our case? </br></br>
2. What sampling `temperature` should we use? Justify.



<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
Your answer here.

1. It will be very efficient because we're using the same system prompt for all generations. The system prompt forms a common prefix across all conversations, allowing vLLM to cache and reuse the computed key-value states for this shared prefix.

2. We should use a low temperature (0.2 for example) because we want consisten and high-quality question generation rather than diverse but potentially noisy outputs. For distillation data generation, we prioritize accuracy and relevance over creativity.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Optimized Generation of Synthetic Questions

Complete the below code with the adequate options (prefix caching and `temperature`)

<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
from vllm import LLM
from vllm import SamplingParams
import os

os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"
llm = LLM(model=path_teacher, gpu_memory_utilization=0.8, max_model_len=5000, enable_prefix_caching=True) # To Complete
sampling_params = SamplingParams(temperature=0.2, max_tokens=400) # To Complete

INFO 11-23 22:18:35 [utils.py:253] non-default args: {'max_model_len': 5000, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-7B-Instruct-AWQ'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

INFO 11-23 22:19:01 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-23 22:19:01 [model.py:1745] Using max model len 5000
INFO 11-23 22:19:10 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

INFO 11-23 22:26:42 [llm.py:352] Supported tasks: ['generate']


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Question Generation</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

We're going to use the llm.chat vLLM api to generate our samples.
Make the adequate code to generate the data and save it in a questions.jsonl file.

Entries of jsonl file should look like:
```json
{
  "id_doc": <wikipedia_article_id>,
  "id_paragraph": <paragraph_dataset_id>,
  "question": <generated_question>,
  "title": <title_wikipedia_article>,
  "paragraph": <text_of_paragraph>
}
```

You should:

1. Complete `def extract_question(generated_text: str) -> str:` to extract the generated question.
2. Complete `def conversation_generator` to output a generator directly ingestible by `llm.chat` api
3. Complete the dataloader and for loop to generate samples by batches of 4
4. Save the generated questions in a `questions.jsonl` file

In [ ]:
from typing import Iterable, Iterator, List
import json
def extract_question(generated_text: str) -> str:

    """Extract the question from the generated text. If the question is not\
    following the right format return None."""

    try:
      j: dict
      # To Complete
      start_idx = generated_text.find('{')
      end_idx = generated_text.rfind('}') + 1
      if start_idx == -1 or end_idx == -1:
        return None

      j = json.loads(generated_text[start_idx:end_idx])
      assert "question" in j
      return j["question"]
    except (json.JSONDecodeError, AssertionError, KeyError):
      return None


def conversation_generator(
    entries: Iterable[dict],
    system_prompt: str
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    for entry in entries:
      conversation: List[dict]
      # To Complete

      user_content = json.dumps({
          "title": entry["title"],
          "paragraph": entry["paragraph"]
      })

      conversation = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_content}
      ]
      yield conversation

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset

ds = load_dataset("EvanD/Lab4_wikiparagraphs")

batch_size = 4
conversations = list(conversation_generator(entries=ds["train"], system_prompt=system_prompt))# To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)


README.md:   0%|          | 0.00/337 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/65.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from tqdm.auto import tqdm
import jsonlines


with jsonlines.open("questions.jsonl", "w") as writer:
    for i, batch in tqdm(enumerate(dataloader)):
        id_paragraph = i * batch_size
        outputs = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for j, output in enumerate(outputs):

          original_idx = id_paragraph + j
          entry = {}
          q = extract_question(output.outputs[0].text)

          if q is not None:
            original_entry = ds["train"][original_idx]

            entry={
                "id_doc": original_entry["id"],
                "id_paragraph": original_idx,
                "question": q,
                "title": original_entry["title"],
                "paragraph": original_entry["paragraph"]
            }
            writer.write(entry)
          else:
            print(f"Question extraction failed for entry {original_idx}")

          # You must verify that the output actually contains a question and write it
          # To Complete

0it [00:00, ?it/s]

INFO 11-23 22:26:47 [chat_utils.py:557] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


## 1.2 - Logprobs Generation

We get to the second part of this distillation where we are interested in distilling the answers logprobs of our teacher model to specialize our 0.5B model to perform Retrieval Augmented Generation (RAG).

First we're going to generate the logprobs with our 7B parameter model.

We'll use the following system prompt:

In [ ]:
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: Complete Conversation Generator</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

Based on the previous code you made to generate questions, make a second one to generate answers and saving logprobs. We updated the sampling parameters of vLLM to return the logprobs and the token ids of the 20 most probable tokens.

As this can result in high quantity of data in practice we're going to store generated conversations in a .jsonl file and generated logprobs and token ids to a hdf5 file (see https://docs.h5py.org/en/stable/ for more information on hdf5).

`save_logprobs_hdf5()` is already implemented for you, and allows to save the logprobs to a .h5 hdf5 file, and increments sequences automatically

The structure of the conversation generator `conversation_generator()` function is implemented, you need to complete it.



In [ ]:
import h5py, os
import numpy as np

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs  = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs",  data=logprobs,  compression="gzip")





<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 6:</br>
When completing conversation_generator we have several ways of sampling.
What are good paragraph sampling strategies we could use to ensure good performance of downstream model?

<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Your answer here.

Good paragraph sampling strategies include:

1. Mixed relevance sampling: Include both relevant paragraphs (containing the answer) and irrelevant ones to teach the model to distinguish between useful and useless context.
2. Progressive difficulty: Start with easy examples (answer directly in context) and gradually introduce harder ones (answer requires reasoning across multiple paragraphs).
3. Negative sampling: Include some paragraphs that don't contain the answer to teach the model when to say "I don't know".
4. Cross-document sampling: Mix paragraphs from different articles to test the model's ability to handle diverse information sources.
<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)  # We add the title to contextualize the paragraph

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]] # We add the title to contextualize the paragraph
        while len(paragraphs) < number_of_paragraphs_in_context:
          # With 50% probability, add a paragraph from the same article

          # Add a paragraph from a different article
          # To Complete

          if random.random() < 0.5 and len(doc_id_to_paragraphs[q_p["id_doc"]]) > 1:
            other_paragraphs = [p for p in doc_id_to_paragraphs[q_p["id_doc"]] if p != paragraphs[0]]

            if other_paragraphs:
              paragraphs.append(random.choice(other_paragraphs))
          else:

            other_doc_ids= [doc_id for doc_id in doc_id_to_paragraphs.keys() if doc_id != q_p["id_doc"]]

            if other_doc_ids:
              random_doc_id = random.choice(other_doc_ids)
              paragraphs.append(random.choice(doc_id_to_paragraphs[random_doc_id]))

        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q_p["question"]}
        ]
        yield conversation



In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Iterable


batch_size = 4

conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt)) # To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

sampling_params = SamplingParams(temperature=0.2, max_tokens=400, logprobs=20)

with jsonlines.open("conversations_rag.jsonl", "w") as writer:
    for batch in tqdm(dataloader):
        out = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for i, b in enumerate(batch):
            text = out[i].outputs[0].text
            b.append({"role": "assistant", "content": text})
            writer.write(b)
        save_logprobs_hdf5("logprobs.h5", [out[i].outputs[0].logprobs for i in range(len(out))])

'''
conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))
'''

  0%|          | 0/50 [00:00<?, ?it/s]

'\nconversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))\n'

In [ ]:
import jsonlines
import random

with jsonlines.open("conversations_rag.jsonl") as reader:
    data = list(reader)

print("Total conversations:", len(data))

# Print 3 random conversations
for conv in random.sample(data, 3):
    print("\n------ Conversation ------")
    for turn in conv:
        print(f"{turn['role'].upper()}: {turn['content'][:200]}")


Total conversations: 200

------ Conversation ------
SYSTEM: You are an assistant for a Retrieval-Augmented Generation (RAG) system.
Answer the question using only the provided documents. If the answer cannot be found in the provided documents, respond that the
USER: What are some examples of exercises that may be included in the beginning of an aikido class?
ASSISTANT: Based on Document 1, some examples of exercises that may be included in the beginning of an aikido class are stretching and breaking falls.

------ Conversation ------
SYSTEM: You are an assistant for a Retrieval-Augmented Generation (RAG) system.
Answer the question using only the provided documents. If the answer cannot be found in the provided documents, respond that the
USER: When was the first election for Aberdeenshire Council held, and when did the new system come into force?
ASSISTANT: The first election for Aberdeenshire Council was held in 1995, and the new system came into force on 1 April 1996.

------ Conve

In [ ]:
test_batch = conversations[:4]
test_out = llm.chat(test_batch, sampling_params=sampling_params)

for o in test_out:
    print("Output text:", o.outputs[0].text[:200])
    print("Has logprobs:", hasattr(o.outputs[0], "logprobs"))


Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Output text: The range of values for albedo is from 0 to 1. A value of 0 represents a body that absorbs all incident radiation (like a black body), while a value of 1 represents a body that reflects all incident r
Has logprobs: True
Output text: According to the paragraph, the albedo for land surfaces at a specific solar zenith angle θi can be approximated by the proportionate sum of two terms.
Has logprobs: True
Output text: According to the Hadley Centre's studies, new forests in high latitudes (e.g., Siberia) tended to be neutral or perhaps warming.
Has logprobs: True
Output text: The total sales tax rate for a diner in Mobile County, Alabama, including the restaurant tax, is 11%.
Has logprobs: True


## 1.3 - KL-Divergence and Distillation

**You should restart the notebook kernel to free the gpu memory from the 7B model that is no longer needed**

Now that we have the generated conversations and their logprobs we can train our 0.5B model to output the same distribution.

The attentive student would have noticed that we have an incomplete representation of the probability distribution over tokens due to only keeping the top-20 logprobs.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 7:</br>
What are the two solutions you can see to approximate full distillation despite only having the top-20 logprobs?

<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Your answer here.

I think there are two solutions:

1. We could apply temperature scaling to both teacher and student distributions before computing KL divergence, which reduces the impact of low-probability tokens.
2. We could focus the distillation loss only on the top tokens from the teacher, effectively treating other tokens as having zero probability. This is reasonable since the teacher's probability mass is concentrated in the top tokens.
<hr style="border:10px solid green"> </hr>
</font></h4>

We supply two functions to help in this implementation:

- `find_subsequence()`that allows to find all the occurences of a token subsequence in a 1-D tensor
- `get_labels()` that allows to expand the top-20 logprobs to the whole vocabulary, implicitly setting prob to zero for other tokens
- `QwenKLDataset`that loads samples from .h5 and .jsonl files, remove problematic inconsistent tokenized examples and outputs samples tokenized for training.

To simplify we will train with a batch size of 1, you can implement gradient accumulation if you wish.

The `PREFIX_ASSISTANT`variable contains the tokens that encode for

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""

    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

In [ ]:
from torch.utils.data import Dataset
import jsonlines
import h5py


PREFIX_ASSISTANT = [198, 151644, 77091, 198]

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:

        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()

        return entry

KL-Divergence is a metric often used to quantify the difference of probability mass between two distributions.
It's not mathematically defined as a distance because of its asymetric nature:

$$D_{KL}(P \parallel Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)}$$

In knowledge distillation, we often minimize the **Kullback–Leibler divergence** between the teacher’s output distribution $P_T$ and the student’s output distribution $P_S$.

$$
D_{KL}(P_T \parallel P_S) = \sum_x P_T(x) \log \frac{P_T(x)}{P_S(x)}
$$
$$
D_{KL}(P_S \parallel P_T) = \sum_x P_S(x) \log \frac{P_S(x)}{P_T(x)}
$$


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 8:</br>
What qualitative difference would it make if we minimize $D_{KL}(P_T \parallel P_S)$ instead of $D_{KL}(P_S \parallel P_T)$? How would it affect the convergence of the student distribution?

<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 8: </b><br>
Your answer here.

Minimizing $D_{KL}(P_T \parallel P_S)$ encourages the student to cover all modes of the teacher's distribution, potentially leading to smoother outputs. Minimizing $D_{KL}(P_S \parallel P_T)$ encourages the student to focus on the dominant modes of the teacher, potentially leading to more confident but less diverse outputs. In practice, the first divergence is preferred for distillation as it better better preserves the teacher's distribution characteristics.
<hr style="border:10px solid green"> </hr>
</font></h4>

**You need to ensure you restarted the kernel and have sufficiently free memory (no 7B model running)**

check with `! nvidia-smi`

In [ ]:
! nvidia-smi

Sun Nov 23 23:02:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----




This code is very basic, and allows you to test a training over the small number of samples you've generated. Due to limitations of Google Colab we cannot go much further, but this first part should have given you the basics on how to perform white-box distillation.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: Complete The Training Code</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>


- Loss function
- gradient accumulation handling



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import gc

torch.cuda.empty_cache()
gc.collect()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
student_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", torch_dtype = torch.float, device_map = "auto") # To Complete
dataset = QwenKLDataset("logprobs.h5", "conversations_rag.jsonl") # To Complete

num_epochs = 2
grad_accum_steps = 16
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])
num_training_steps = num_epochs * len(loader) // grad_accum_steps
scheduler = get_linear_schedule_with_warmup(optimizer, 0, num_training_steps)
loss_fn = torch.nn.KLDivLoss(reduction='batchmean') # To Complete

student_model.gradient_checkpointing_enable()

for epoch in range(num_epochs):
    total_loss = 0
    student_model.train()
    for i, batch in enumerate(loader):

      with torch.autocast(device_type = 'cuda', dtype = torch.float16):
        input_ids = batch["inputs"]["input_ids"].to(device)
        attention_mask = batch["inputs"].get("attention_mask")
        if attention_mask is not None:
          attention_mask = attention_mask.to(device)

        labels = batch["labels"].to(device)

        if attention_mask is not None:
          outputs = student_model(input_ids, attention_mask=attention_mask)
        else:
          outputs = student_model(input_ids)

        answer_start = batch["idx_subseq"]
        answer_end = answer_start + batch["seq_len"]
        student_logits = outputs.logits[0, answer_start:answer_end, :]

        temperature = 1.0
        student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
        teacher_probs = F.softmax(labels /temperature, dim=-1)

        loss = loss_fn(student_log_probs, teacher_probs)
        loss = loss / grad_accum_steps
        loss.backward()

        total_loss += loss.item()* grad_accum_steps
        # To Complete
        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            current_loss = total_loss / (i+1)
            print (f"Epoch {epoch+1}, step {i+1}/{len(loader)}: loss = {current_loss:.4f}")

    epoch_avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}: average loss = {epoch_avg_loss:.4f}")

student_model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")

`torch_dtype` is deprecated! Use `dtype` instead!


0it [00:00, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch 1, step 16/200: loss = 0.4528
Epoch 1, step 32/200: loss = 0.7642
Epoch 1, step 48/200: loss = 0.6969
Epoch 1, step 64/200: loss = 0.7221
Epoch 1, step 80/200: loss = 0.7500
Epoch 1, step 96/200: loss = 0.7573
Epoch 1, step 112/200: loss = 0.7170
Epoch 1, step 128/200: loss = 0.6964
Epoch 1, step 144/200: loss = 0.6751
Epoch 1, step 160/200: loss = 0.6718
Epoch 1, step 176/200: loss = 0.6909
Epoch 1, step 192/200: loss = 0.6750
Epoch 1, step 200/200: loss = 0.6705
Epoch 1: average loss = 0.6705
Epoch 2, step 16/200: loss = 0.3187
Epoch 2, step 32/200: loss = 0.2894
Epoch 2, step 48/200: loss = 0.2831
Epoch 2, step 64/200: loss = 0.2809
Epoch 2, step 80/200: loss = 0.3134
Epoch 2, step 96/200: loss = 0.2990
Epoch 2, step 112/200: loss = 0.2962
Epoch 2, step 128/200: loss = 0.2901
Epoch 2, step 144/200: loss = 0.2892
Epoch 2, step 160/200: loss = 0.2972
Epoch 2, step 176/200: loss = 0.2919
Epoch 2, step 192/200: loss = 0.2837
Epoch 2, step 200/200: loss = 0.2799
Epoch 2: average lo

('finetuned_model/tokenizer_config.json',
 'finetuned_model/special_tokens_map.json',
 'finetuned_model/chat_template.jinja',
 'finetuned_model/vocab.json',
 'finetuned_model/merges.txt',
 'finetuned_model/added_tokens.json',
 'finetuned_model/tokenizer.json')

# Part 2 - Retrieval Augmented Generation (RAG)

In this section, we will discuss the concept of **Retrieval-Augmented Generation (RAG)** — a framework that combines **information retrieval** and **language generation**. RAG enables language models to access **external knowledge sources** at inference time, reducing hallucinations and improving factual accuracy.

We will explore how to:
- Build and index a **Vector database** from a corpus (here: Wikipedia sample).
- Retrieve the most relevant documents given a query using **embedding-based similarity**.
- Integrate retrieval results into the **generation pipeline** to produce context-aware answers.


In [ ]:
# restart the session and run
!pip -q install chromadb==0.4.22
!pip -q install "numpy<2.0" --force-reinstall
!pip -q install datasets==2.21.0 pandas==2.2.2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-o

In [ ]:
import os
import json
import random
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel

In [ ]:
# --- Configuration ---
SEED = 42
NUM_ROWS = 1000
DATA_PATH = "wikipedia_20231101_en_1000.csv"

if os.path.exists(DATA_PATH):
    print(f"✅ Found existing dataset at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    print("⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...")
    random.seed(SEED)

    # Load the Wikipedia dataset
    stream_ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True
    )

    buffered_stream = stream_ds.shuffle(seed=SEED, buffer_size=200_000)

    sampled = []
    for ex in buffered_stream:
        try:
            if int(ex["id"]) % 2 == 0:
                sampled.append(ex)
            if len(sampled) >= NUM_ROWS:
                break
        except:
            continue

    # Create DataFrame
    df = pd.DataFrame(sampled)[["id", "url", "title", "text"]]
    df.to_csv(DATA_PATH, index=False)
    print(f"💾 Dataset saved to {DATA_PATH}")


⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

💾 Dataset saved to wikipedia_20231101_en_1000.csv


In [ ]:
#Display Basic Info ---
print("Sampled shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

Sampled shape: (1000, 4)
Columns: ['id', 'url', 'title', 'text']


,id,url,title,text
0,64741026,https://en.wikipedia.org/wiki/Indrani%20Perera,Indrani Perera,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,66847582,https://en.wikipedia.org/wiki/August%20Laur,August Laur,August Laur (9 October 1886 Vana-Põltsamaa Par...
2,66467526,https://en.wikipedia.org/wiki/Daniele%20Solcia,Daniele Solcia,Daniele Solcia (born 7 March 2001) is an Itali...
3,65913988,https://en.wikipedia.org/wiki/Eric%20Takabatake,Eric Takabatake,Eric Takabatake (born 9 January 1991) is a Bra...
4,64723650,https://en.wikipedia.org/wiki/Nafissath%20Radji,Nafissath Radji,Nafissath Radji (born 2 August 2002 in Porto-N...


In [ ]:
# Visualize Random Wikipedia Articles ---

NUM_EXAMPLES = 3  # number of random samples to show
samples = df.sample(NUM_EXAMPLES, random_state=random.randint(0, 10000))

for _, row in samples.iterrows():
    display(HTML(f"""
    <hr style="border:2px solid #ccc">
    <h3><b>Title:</b> {row['title']}</h3>
    <p><b>URL:</b> <a href="{row['url']}" target="_blank">{row['url']}</a></p>
    <p style="text-align: justify;"><b>Text:</b><br>{row['text']}</p>
    """))


### **Document Chunking**

The first step in building a RAG pipeline is **chunking**, where large documents are divided into smaller, semantically coherent pieces.  
Chunking allows the retriever to work on manageable text segments instead of entire documents, improving retrieval precision and reducing computational load.  


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Naive Fixed-Length Chunking  
Split each document into overlapping fixed-length chunks to prepare text for retrieval.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:

def text_splitting(text, chunk_length=300, chunk_overlap=100):
    """
    Splits text into fixed-length chunks with overlap.
    """
    out = []
    stride = chunk_length - chunk_overlap## FILL THE GAP: define the stride as the effective step between chunks
    for i in range(0, len(text), stride):
        chunk = text[i:i+ chunk_length]## FILL THE GAP: extract a substring of size 'chunk_length' starting at 'i'
        out.append(chunk)
    return out

# Apply to all documents
df["naive_chunks"] = df["text"].apply(lambda t: text_splitting(t, chunk_length=300, chunk_overlap=100))

num_chunks = df["naive_chunks"].apply(len)
print(f"Average number of chunks per document: {num_chunks.mean():.2f}")
print(f"Total number of chunks: {num_chunks.sum()}")

example_idx = 0
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]["title"])
print("Original length:", len(df.iloc[example_idx]["text"]))
print("Number of chunks:", len(df.iloc[example_idx]["naive_chunks"]))
print("\nFirst 2 chunks:\n")
for i, c in enumerate(df.iloc[example_idx]["naive_chunks"][:2]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

Average number of chunks per document: 11.50
Total number of chunks: 11498

--- Example document ---
Title: Indrani Perera
Original length: 3012
Number of chunks: 16

First 2 chunks:

Chunk 1:
Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer. Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls 
--------------------------------------------------------------------------------
Chunk 2:
la Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls in the family. Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian. She has one elder sister, Mallika and one younger sister, Iranganie. Indrani studied at  Presbyteri
--------------------------------------------------------------------------------


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Paragraph-Aware Chunking  
Implement a smarter chunking strategy by using the ('.') as a boundary to split text into sentences or short paragraphs, then regroup them until reaching the desired chunk length.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def text_splitting_paragraph(text, chunk_length=300):
    """
    Splits text by sentences/paragraphs (using '.' as boundary)
    and groups them until reaching the desired chunk length.
    """
    out = []
    paragraph_list = text.split('.')## FILL THE GAP: split the text into smaller parts using '.' as a separator
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += '. '+par## FILL THE GAP: extend the ongoing chunk with the next segment
            length += len(par) + 2## FILL THE GAP: increment the total length accordingly
        else:
            if len(current_text) != 0:
                out.append(current_text)## FILL THE GAP: store the current completed chunk before starting a new one)
            current_text = par## FILL THE GAP: initialize a new chunk with the current paragraph
            length = len(par)## FILL THE GAP: reset the chunk length counter
    if len(current_text) > 0:
        out.append(current_text)## FILL THE GAP: add the last remaining chunk to the list)
    return out


# Apply to all documents
df["paragraph_chunks"] = df["text"].apply(lambda t: text_splitting_paragraph(t, chunk_length=300))

# Compute stats
num_chunks_par = df["paragraph_chunks"].apply(len)
print(f"Average number of paragraph-based chunks per document: {num_chunks_par.mean():.2f}")
print(f"Total number of paragraph-based chunks: {num_chunks_par.sum()}")

# Example comparison
example_idx = 432 #Check out other examples
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]['title'])
print("Original length:", len(df.iloc[example_idx]['text']))
print(f"Character based chunks: {len(df.iloc[example_idx]['naive_chunks'])}")
print(f"Paragraph based chunks: {len(df.iloc[example_idx]['paragraph_chunks'])}")

print("\nParagraph chunk preview:\n")
for i, c in enumerate(df.iloc[example_idx]['paragraph_chunks'][:6]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")


Average number of paragraph-based chunks per document: 8.94
Total number of paragraph-based chunks: 8940

--- Example document ---
Title: John Ballantine (banker)
Original length: 12204
Character based chunks: 62
Paragraph based chunks: 50

Paragraph chunk preview:

Chunk 1:
John Ballantine (1743–1812), was a Scottish merchant and banker and one of the greatest friends, admirers and closest confidants of Robert Burns
--------------------------------------------------------------------------------
Chunk 2:
  Significantly Ballantine gave the poet advice on the selection of poems for his First Kilmarnock Edition as well as being asked for his opinion on the bard's poems. 

Life and character
John was born in Ayr to William Ballantine, a baillie in Ayr and his mother was Elizabeth Bowman
--------------------------------------------------------------------------------
Chunk 3:
 He was a merchant and a Banker and in 1787 he became the Provost of Ayr, during which time he helped establish Ay

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:  
In the paragraph-aware chunking method above, we simply split Wikipedia text using the '.' delimiter to approximate sentence boundaries.  
Discuss whether this is an effective strategy for creating meaningful chunks in a RAG system.
Propose one or more improved chunking strategies that could better capture document structure — and you may include code snippets to justify or demonstrate your approach.  
<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Your answer here.

Using '.' as the only delimiter is not very effective, since it doesn't handle abbreviations (such as "dr." or "etc."). Another issue is that it ignores other sentence boundaries like '!' or '?'. Finally, it doesn't consider paragraph structure.

Now, I'll show some strategies to improve chunking:

- The firt option is to use embeddings to find natural break points based on topic changes.
- The second one is the NLTK sentence tokenizer, which is shown below.
<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
##NLTK sentence tokenizer : This is a more robust sentence boundary detection method

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def better_chunking(text, chunk_length=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_length:
            current_chunk += " " + sentence
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: Consider a scenario where you want to perform RAG on source code (e.g., Python files, Java classes) instead of natural language text. Would the chunking methods demonstrated above (character-based and sentence/paragraph based with boundaries) work effectively for code? Explain why or why not, and describe how you would approach chunking source code to maintain semantic coherence. What specific characteristics of code structure would you need to consider?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
Your answer here.

The demonstrated chunking methods would not work effectively for source code, due to these reasons:

- Code has structured syntax (functions, classes, blocks) rather than natural language texts.
- Splitting at '.' or using fixed lengths would easily break code syntax and semantics.
- Code is mainly defined by logical units.

I would suggest these as better approach for code chunking:

- We could use abstract syntax trees to identify logical units, such as functions, classes, methods, etc.
- Probably it would be a good idea to keep related functions with its helper functions.
- Besides, we could include the libraries used by the chunk.
- We could also keep comments and docstrings with their relevant code.
<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
# Saving chunks ---

# Flatten chunks into a new DataFrame
records = []
for _, row in df.iterrows():
    doc_id = row["id"]
    title = row["title"]
    url = row["url"]
    for i, chunk in enumerate(row["paragraph_chunks"]):
        records.append({
            "doc_id": doc_id,
            "title": title,
            "url": url,
            "chunk_id": f"{doc_id}_chunk_{i}",
            "chunk_text": chunk.strip()
        })

# Create the flattened chunks DataFrame
chunks_df = pd.DataFrame(records)
print(f"Total chunks: {len(chunks_df)}")
print(f"Average chunk length: {chunks_df['chunk_text'].apply(len).mean():.2f} characters\n")

# Show example
print("Example rows:")
display(chunks_df.head())

chunks_df.to_csv("wikipedia_chunks.csv", index=False)
print("Chunks saved to 'wikipedia_chunks.csv'")
print(f"Total chunks: {len(chunks_df)}")


Total chunks: 8940
Average chunk length: 245.12 characters

Example rows:


,doc_id,title,url,chunk_id,chunk_text
0,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_0,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_1,Early life \nShe was born on 15 February in Bo...
2,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_2,Indrani studied at Presbyterian Girls School ...
3,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_3,She met him during the production of his song ...
4,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_4,After that he studied A/L at the Royal Institu...


Chunks saved to 'wikipedia_chunks.csv'
Total chunks: 8940


## <b>Part II: Embedding</b>

After chunking our documents, the next step is to convert text chunks into vector representations (embeddings). These embeddings capture the semantic meaning of the text in a high-dimensional space, allowing us to measure similarity between chunks and queries mathematically.

We will use **sentence-transformers/all-MiniLM-L6-v2**, a compact and efficient embedding model that produces 384-dimensional embeddings for English text. This model offers a strong balance between performance and computational efficiency, making it well-suited for our RAG pipeline.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill in the <code>embed()</code> function to encode text chunks and generate normalized embeddings using <code>sentence-transformers/all-MiniLM-L6-v2</code>.  
Then, apply it to all documents in <code>chunks_df</code> and store the results.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
# --- 2.3 Embedding Generation ---
# Load model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"Loaded embedding model: {model_name}")


# --- Define embedding function ---
def embed(text_list, doc_type="document"):
    """
    Encodes a list of texts and returns normalized embeddings.
    """
    encoded = tokenizer(
        [f"search_{doc_type}: {t}" for t in text_list],
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoded)## FILL THE GAP: forward pass through the model to obtain hidden states
        token_embeddings = output.last_hidden_state## FILL THE GAP: extract the last hidden state from the output
        attention_mask = encoded['attention_mask']
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        pooled = torch.sum(token_embeddings*input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min= 1e-9)
        pooled = pooled / sum_mask## FILL THE GAP: aggregate token embeddings (e.g., by summing along sequence dimension)
        pooled = F.normalize (pooled, p = 2, dim=1)## FILL THE GAP: apply L2 normalization along the embedding dimension
    return pooled.cpu()



tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2


In [ ]:
# --- Test with one example ---
sample_text = ["Artificial intelligence is transforming the world."]
sample_emb = embed(sample_text)
print(f"Sample embedding shape: {sample_emb.shape}")

# --- Apply to all chunks ---
print(f"\nGenerating embeddings for {len(chunks_df)} chunks...")

emb_list = []
for i in tqdm(range(0, len(chunks_df), 32)):
    batch = chunks_df["chunk_text"].iloc[i:i+32].tolist()
    emb = embed(batch, doc_type="document")
    emb_list.append(emb)

chunk_embeddings = torch.cat(emb_list, dim=0).numpy()
chunks_df["embedding"] = list(chunk_embeddings)

print("\nEmbeddings generated and added to DataFrame.")
print(chunks_df[["chunk_id", "title", "embedding"]].head())

Sample embedding shape: torch.Size([1, 384])

Generating embeddings for 8940 chunks...


100%|██████████| 280/280 [00:13<00:00, 21.40it/s]



Embeddings generated and added to DataFrame.
           chunk_id           title  \
0  64741026_chunk_0  Indrani Perera   
1  64741026_chunk_1  Indrani Perera   
2  64741026_chunk_2  Indrani Perera   
3  64741026_chunk_3  Indrani Perera   
4  64741026_chunk_4  Indrani Perera   

                                           embedding  
0  [-0.100748904, -0.010456987, -0.0718354, -0.05...  
1  [-0.010396938, 0.007939495, -0.11834743, 0.093...  
2  [-0.08076134, -0.04774314, -0.023494773, -0.01...  
3  [-0.107841216, 0.061594483, -0.05191377, 0.055...  
4  [-0.06349757, 0.005113634, -0.011235888, 0.020...  


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: In most retrieval systems, embeddings are represented as fixed-size vectors. Can you think of a way to design embeddings that can flexibly adjust their size or level of detail while still preserving meaningful similarity between representations? How could such an approach benefit Retrieval-Augmented Generation (RAG) systems in practice, particularly for improving efficiency or adapting to different computational budgets?

<hr style="border:10px solid red"> </hr>
<i>Hint:</i> You can find the answer in the paper <a href="https://arxiv.org/pdf/2205.13147" target="_blank">Matryoshka Representation Learning</a>.


</font></h4>

The Matryoshka Representation Learning (MRL) is a good way to flexibly adjust the embeddings size. It does it by learning, nested, multi-granularity representatoins within a single high-dimensional vector. This single vector contains coarse-to-fine information across its dimensions.The first m dimensions form a representation that is as accurate as an independently trained m-dimensional model.

Some benefits for RAG systems I noticed are these ones:

- Efficient retrieval via Adaptive Search, since the low-dimensional embedding allows to quickly shortlist a large set of candidates, to re-rank the shortlist using a higher-dimensional embedding for precision, reducing the computational cost of similarity search.

- Dynamic adaptation to compute budgets, because the MRL allows the same model to serve multiple resource constraints without retraining or storing multiple models. In the context of RAG system, this would mean to use smaller embeddings for low-resource settings and full embeddings for high-resource settings.


### **Building a Simple Vector Database**

After generating embeddings for all our document chunks, the next step is to **store** them in a structure that allows fast similarity search.  
In this section, we will build a **simple in-memory vector database** using PyTorch tensors.  
Each entry in the database will correspond to a text chunk and its embedding, enabling efficient retrieval based on vector similarity.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill in the code to populate the database by computing embeddings for all text chunks in <code>chunks_df</code> using the <code>embed()</code> function.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:

def populate_database(chunks_df, batch_size=16):
    """
    Populates a vector database from precomputed chunks_df.
    """
    n_chunks = len(chunks_df)

    sample_emb = embed([chunks_df["chunk_text"].iloc[0]])## FILL THE GAP: compute one sample embedding to infer the output dimension
    output_dim = sample_emb.shape[1]## FILL THE GAP: extract the embedding dimension from the sample

    vectorial_database = torch.zeros((n_chunks, output_dim))## FILL THE GAP: initialize an empty tensor to store all embeddings
    chunk_list = chunks_df["chunk_text"].tolist()

    print(f"Populating vector database with {n_chunks} chunks...")

    n = 0
    for i in range(0, n_chunks, batch_size):
        batch = chunk_list[i: i +batch_size]## FILL THE GAP: select a batch of chunk texts
        embeddings = embed(batch)## FILL THE GAP: compute embeddings for the current batch
        vectorial_database[n:n + len(batch)] = embeddings## FILL THE GAP: store embeddings in the tensor
        n += len(batch)

    return chunk_list, vectorial_database



In [ ]:
#Build Vector Database
chunk_list, vectorial_database = populate_database(chunks_df)

print("\n✅ Vector database successfully built.")
print(f"Total stored chunks: {len(chunk_list)}")
print(f"Database tensor shape: {tuple(vectorial_database.shape)}")

Populating vector database with 8940 chunks...

✅ Vector database successfully built.
Total stored chunks: 8940
Database tensor shape: (8940, 384)


In [ ]:
#Save vector databse
os.makedirs("vector_db", exist_ok=True)

# Save tensor + chunk list
torch.save(vectorial_database, "vector_db/vectorial_database.pth")

with open("vector_db/chunk_list.json", "w", encoding="utf-8") as f:
    json.dump(chunk_list, f, indent=4, ensure_ascii=False)

print("✅ Saved:")
print(" - vector_db/vectorial_database.pth")
print(" - vector_db/chunk_list.json")

✅ Saved:
 - vector_db/vectorial_database.pth
 - vector_db/chunk_list.json


In [ ]:
# Load the database
vectorial_database = torch.load("vector_db/vectorial_database.pth", map_location=device)
vectorial_database.requires_grad_(False)

with open("vector_db/chunk_list.json", "r", encoding="utf-8") as f:
    chunk_list = json.load(f)

print(f"✅ Loaded {len(chunk_list)} chunks.")
print(f"Database shape: {vectorial_database.shape}\n")

# Inspect first few entries
for i, embedding_vector in enumerate(vectorial_database[:5]):
    print(f"Vector {i} → {embedding_vector[:5]}")
    print(f"Text snippet: {chunk_list[i][:300]}\n")


✅ Loaded 8940 chunks.
Database shape: torch.Size([8940, 384])

Vector 0 → tensor([-0.1007, -0.0105, -0.0718, -0.0592, -0.0365], device='cuda:0')
Text snippet: Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer.  Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio"

Vector 1 → tensor([-0.0104,  0.0079, -0.1183,  0.0933, -0.0443], device='cuda:0')
Text snippet: Early life 
She was born on 15 February in Borella, and is the second of three girls in the family.  Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian.  She has one elder sister, Mallika and one younger sister, Iranganie

Vector 2 → tensor([-0.0808, -0.0477, -0.0235, -0.0122, -0.0852], device='cuda:0')
Text snippet: Indrani studied at  Presbyterian Girls School in Regent Street.  She studied Kandyan Dancing in the school.  Her sister Mallika has been singing since 1965 where she was the pla

#### **Defining Similarity Metrics for Retrieval**

After populating our vector database with embeddings, the next step in a RAG pipeline is to define a *similarity metric* to measure how close two vectors are in the embedding space.  
Common metrics include **dot product**, **L2 distance**, and **cosine similarity**.  

In most retrieval systems, cosine similarity is preferred because it measures the *angle* between two vectors rather than their magnitude, allowing comparison based purely on semantic direction instead of scale.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill in the code to implement the <code>cosine_similarity()</code> function.  
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def cosine_similarity(query_embeddings, doc_embeddings):
    """
    Computes cosine similarity between query and document embeddings
    using manual normalization.
    """
    query_magnitudes = torch.norm(query_embeddings, dim=1, keepdim = True)## FILL THE GAP: calculate vector length for each query
    normalized_queries = query_embeddings / query_magnitudes## FILL THE GAP: normalize queries using their magnitudes

    doc_magnitudes = torch.norm(doc_embeddings, dim=1, keepdim= True)## FILL THE GAP: calculate vector length for each document
    normalized_docs = doc_embeddings / doc_magnitudes## FILL THE GAP: normalize documents using their magnitudes

    similarity_matrix = torch.mm(normalized_queries, normalized_docs.T)## FILL THE GAP: perform dot product between normalized queries and documents

    return similarity_matrix


# --- Example test ---
query_embeddings = embed([
    "What is t-SNE?",
    "Who is Laurens van der Maaten?"
], "query")

doc_embeddings = embed([
    "t-SNE is a dimensionality reduction algorithm created by Laurens van der Maaten."
], "document")

with torch.no_grad():
    sim_cos = cosine_similarity(query_embeddings, doc_embeddings)

print("🔍 Example cosine similarity scores:\n", sim_cos)

🔍 Example cosine similarity scores:
 tensor([[0.6621],
        [0.4333]])


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill in the code to complete the <code>retrieve()</code> function.  
It should encode the input query using <code>embed()</code>, compute similarity with all vectors in <code>vectorial_database</code>, and return the top-<i>k</i> most similar text chunks.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def retrieve(query,
             vectorial_database=vectorial_database,
             chunk_list=chunk_list,
             topk=5,
             verbose=False):
    """
    Retrieves top-k most similar chunks to a query using dot-product similarity.
    """
    with torch.no_grad():
        query_embedding = embed([query], "query")## FILL THE GAP: encode the input query using the embed() function
        query_embedding = query_embedding.to(device)## FILL THE GAP: move the query embedding to the correct device
        similarity_scores = torch.mm(query_embedding, vectorial_database.T)## FILL THE GAP: compute similarity between query and database embeddings
        topk_results = torch.topk(similarity_scores, k= topk, dim=1)## FILL THE GAP: extract the top-k highest similarity scores and indices

        if verbose:
            for score, idx in zip(topk_results.values[0], topk_results.indices[0]):
                print(f"\nScore: {score:.4f}")
                print(f"Chunk:\n{chunk_list[idx][:500]}\n{'-'*80}")

        retrieved_chunks = [chunk_list[idx] for idx in topk_results.indices[0]]## FILL THE GAP: select text chunks corresponding to the top-k indices
        return "\n\n".join(retrieved_chunks)## FILL THE GAP: return concatenated retrieved chunks as a single string


In [ ]:
# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)



Score: 0.5731
Chunk:
Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016.  Boria defeated former Miami-Dade School Board member Frank Bolaños in the 2012 elections, obtaining 54% of the vote
--------------------------------------------------------------------------------

Score: 0.5610
Chunk:
(1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (1776–1813)
Sede vacante (1813–1817)
 Aloisio San Vitale (1817–1836)
 Giovanni Tommaso Neuschel (1836–1843)
 Pier Grisologo Basetti (1843–1857)
Sede vacante (16 June 1857 – 20 June 1859)
 Francesco Ben
--------------------------------------------------------------------------------

Score: 0.5299
Chunk:
Biography 
Boria, who 

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 4: There are retrieval methods like BM25 that rely on lexical overlap between the query and documents, and others based on dense embeddings that capture semantic similarity beyond exact word matches. Explain how these two approaches differ in how they represent and compare text. Then, discuss how a hybrid retrieval strategy combining both can overcome their respective limitations and improve retrieval performance in RAG systems. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
Your answer here.

BM25 relies on exact word matches and term frequency, making it excellent for keyword-based queries but poor for semantic understanding or synonym handling.
The other menntioned, the dense embeddings, capture meaning beyond exact words, handling synonyms and related concepts well but sometimes missing exact keyword matches.

A hybrid approach would benefit aspects like:

- Recall improvement, since it combines strengths of both methods.
- Query understanding, handling both keyword-heavy and conceptual queries.
- It would also benefit the robustness, since it would be less sensitive to vocabulary mismatches.
<hr style="border:10px solid green"> </hr>
</font></h4>


In real-world RAG systems, instead of manually storing and comparing vectors, we rely on **vector databases** such as **ChromaDB**, which are optimized for efficient **similarity search**, **indexing**, and **retrieval** at scale.  

These databases provide:
- Fast nearest-neighbor search (e.g., using HNSW graphs)  
- Persistent storage for millions of embeddings  
- Built-in support for different similarity metrics (cosine, L2, inner product)  


In [ ]:
import chromadb
from chromadb.config import Settings

# --- Initialize ChromaDB client ---
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset ensures a clean state
chroma_client.reset()

# --- Create a collection ---
# You can choose the similarity metric: "cosine", "l2", or "ip" (inner product)
collection_name = "wikipedia_chunks"
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine similarity works best for normalized embeddings
)

print(f"✅ Created collection: {collection_name}")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Created collection: wikipedia_chunks


In [ ]:
# Prepare the data
ids = chunks_df["chunk_id"].tolist()
embeddings = chunks_df["embedding"].tolist()
documents = chunks_df["chunk_text"].tolist()

# Ensure all embeddings are plain Python lists
embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]

# Add useful metadata for inspection
metadatas = [
    {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "url": row["url"]
    }
    for _, row in chunks_df.iterrows()
]

# Add to the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

print(f"\n✅ Added {collection.count()} chunks to the collection")
print(f"Collection metadata: {collection.metadata}")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given



✅ Added 8940 chunks to the collection
Collection metadata: {'hnsw:space': 'cosine'}


In [ ]:
# Encode the query using our embed() function
query = "When was Luigi Boria born?"
query_embedding = embed([query], doc_type="query")[0].tolist()  # get single vector as list

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3 # number of retrieved results
)

# Display results
print(f"🔎 Query: {query}\n" + "=" * 80)
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"Title: {results['metadatas'][0][i]['title']}")
    print(f"Similarity score: {1 - results['distances'][0][i]:.4f}")  # cosine distance → similarity
    print(f"Text: {results['documents'][0][i][:300]}...")
    print("-" * 80)


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


🔎 Query: When was Luigi Boria born?

Result 1:
Title: Luigi Boria
Similarity score: 0.5731
Text: Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016.  Boria defeated former Miami-Dade School Board member Frank Bolaños in the 2012 elections, obtaining 54% of the vote...
--------------------------------------------------------------------------------

Result 2:
Title: Roman Catholic Diocese of Fidenza
Similarity score: 0.5610
Text: (1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (...
--------------------------------------------------------------------------------

Result 3:
Title: Luigi Boria
Similarity score: 0.5299
Text: Biography 
Boria, who is also an evangeli

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5: </b><br>
ChromaDB and other vector databases often rely on Hierarchical Navigable Small World (HNSW) graphs to perform efficient approximate nearest neighbor search.  
Explain how the HNSW algorithm organizes data to enable fast and accurate retrieval in high-dimensional spaces.  
Why is this structure particularly effective for large-scale embedding collections compared to brute-force search?

<hr style="border:10px solid red"> </hr>
<i>Reference:</i> <a href="https://arxiv.org/pdf/1603.09320" target="_blank">Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs</a>
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
Your answer here.

The HNSW algorithm organizes data in a way that enables fast and accurate approximate nearest neighbor search in high-dimensional spaces through a multi-layer graph structure inspired by navigable small-world networks and skip lists.
HNSW builds a hierarchical graph with multiple layer, where each element is assigned to a randomly chosen maximum layer l using an exponentially decaying probability distribution. The top layers contain only a few elements with long-range links, while the bottom layer contains all elements with short-range links, so that it mimics a skip list but replaces linked lists with proximity graphs.

It is effective because compared to brute-force since it has a logarithmic search complexity $O(log N)$ while the BFS has $O(N)$. Besides it has a efficient handling of high-dimensional data.
<hr style="border:10px solid green"> </hr>
</font></h4>


### <b> Two-Stage Retrieval: Dense Retrieval + Reranker</b>

In RAG, the first retrieval step often returns passages that are similar in meaning but not always the most relevant.  
A **reranker** fixes this by re-evaluating the top retrieved chunks using a stronger model that jointly reads the query and each document to assign a more accurate relevance score.


In [ ]:
from sentence_transformers import CrossEncoder

# Load reranker model
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL_NAME)

def retrieve_with_reranker(query, collection, initial_k=5, final_k=3):
    """
    Retrieves and reranks candidate documents for a given query.
    Returns both the initial dense results and reranked results.
    """
    query_embedding = embed([query], doc_type="query")[0].tolist()
    candidates = collection.query(query_embeddings=[query_embedding], n_results=initial_k)

    docs = candidates["documents"][0]
    metas = candidates["metadatas"][0]
    dense_scores = [(1 - s) for s in candidates["distances"][0]]

    pairs = [(query, d) for d in docs]
    ce_scores = reranker.predict(pairs)

    reranked = [
        {
            "title": metas[i].get("title", ""),
            "url": metas[i].get("url", ""),
            "text": docs[i],
            "dense_score": dense_scores[i],
            "rerank_score": float(ce_scores[i]),
        }
        for i in range(len(docs))
    ]
    reranked.sort(key=lambda x: x["rerank_score"], reverse=True)

    return docs, metas, dense_scores, reranked[:final_k]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
# Example usage
query = "Where did Luigi Boria study?" #Try other queries too
docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

# --- Display initial dense retrieval ---
print(f"\nInitial dense retrieval (Top 5):\n" + "=" * 80)
for i, (d, s, m) in enumerate(zip(docs, dense_scores, metas), 1):
    print(f"{i}. {m.get('title', '')}  |  Dense similarity: {s:.4f}")
    print(f"Text: {d[:300].replace('\n', ' ')}")
    print("-" * 80)

# --- Display top reranked results ---
print(f"\nAfter Cross-Encoder Reranking (Top 3):\n" + "=" * 80)
for i, item in enumerate(top_reranked, 1):
    print(f"{i}. {item['title']}")
    print(f"Dense similarity: {item['dense_score']:.4f} | Reranker score: {item['rerank_score']:.4f}")
    print(f"Text: {item['text'][:300].replace('\n', ' ')}")
    print("-" * 80)


Initial dense retrieval (Top 5):
1. Luigi Boria  |  Dense similarity: 0.5119
Text: Biography  Boria, who is also an evangelical Christian pastor, was born in Caracas in 1958 to Italian parents who emigrated to that country.  He studied accounting at the Andrés Bello Catholic University, a private institution, in 1982
--------------------------------------------------------------------------------
2. Luigi Boria  |  Dense similarity: 0.5076
Text: Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016.  Boria defeated former Miami-Dade School Board member Frank Bolaños in the 2012 elections, obtaining 54% of the vote
--------------------------------------------------------------------------------
3. Luigi Boria  |  Dense similarity: 0.5002
Text: In 1989 he left Venezuela to settle in Doral, Florida, where he currently lives.  Boria has run various companies in both Venezuela and the United States since the 1980s

![Bi-Encoder vs Cross-Encoder Architecture](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6:  
The figure above compares a Bi-Encoder and a Cross-Encoder architecture.  
Rerankers such as <code>cross-encoder/ms-marco-MiniLM-L-6-v2</code> use the second approach, jointly encoding the query and document through a single transformer.  
Why does this joint encoding typically yield higher retrieval precision, and why is it applied as a second-stage reranker instead of being used directly for large-scale retrieval?
<hr style="border:10px solid red"> </hr>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

Cross-encoders, which jointly encode the query and document through a single transformer model, typically yield higher retrieval precision because they enable deep, interactive attention between all tokens in the query and document. Unlike bi-encoders that process queries and documents separately and rely on compressed embeddings for similarity computation, cross-encoders allow every token in the query to directly attend to every token in the document. This full interaction captures nuanced semantic relationships, contextual dependencies, and subtle relevance cues that might be lost in the independent encoding process of bi-encoders. As a result, cross-encoders can more accurately assess relevance, especially for complex or ambiguous queries, leading to superior precision in identifying the most pertinent documents.

However, cross-encoders are applied as second-stage rerankers rather than for direct large-scale retrieval due to their computational inefficiency. Jointly encoding each query-document pair requires processing the entire concatenated text through the transformer, which is computationally expensive and scales poorly with corpus size. For a collection with millions of documents, this would involve billions of operations per query, making real-time retrieval infeasible. In contrast, first-stage retrievers like bi-encoders or keyword-based methods use precomputed document embeddings or inverted indexes to quickly narrow down a small candidate set (e.g., top-100 results). The cross-encoder then reranks this manageable subset, combining the speed of scalable retrieval with the precision of deep cross-attention without overwhelming computational costs.

### **Integrating Retrieved Context into the LLM’s Prompt**

Now that we can retrieve and rerank the most relevant document chunks,  
we integrate them directly into the **language model’s prompt**.  
This step allows the model to **ground its answer on factual context** rather than relying solely on internal knowledge —  
thereby improving accuracy and reducing hallucinations.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
gen_tok = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

def generate(msg, max_new_tokens=128, temperature=0.2):
    messages = [{"role": "user", "content": msg}]
    inputs = gen_tok.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    outputs = gen_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        eos_token_id=gen_tok.eos_token_id,
        pad_token_id=gen_tok.pad_token_id if gen_tok.pad_token_id is not None else gen_tok.eos_token_id
    )

    return gen_tok.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()


# ============================================================
# NO-RAG vs WITH RAG
# ============================================================
query = "When was Luigi Boria born?" #Try other queries

print("ORIGINAL PROMPT\n" + "=" * 60)
print(query)

print("\nANSWER WITHOUT RAG\n" + "=" * 60)
print(generate(query))

docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

context = "\n\n".join(h["text"] for h in top_reranked)[:1600]
rag_prompt = (
    f"Use only the context to answer. If unknown, say you don't know.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\nAnswer:"
)

print("\nAUGMENTED PROMPT\n" + "=" * 60)
print(rag_prompt)

print("\nANSWER WITH RAG\n" + "=" * 60)
print(generate(rag_prompt))


ORIGINAL PROMPT
When was Luigi Boria born?

ANSWER WITHOUT RAG
Luigi Boria was born on April 15, 1960. He is an Italian-American writer and filmmaker known for his work in the genres of science fiction, fantasy, and horror. His most famous works include "The Last Man" (2004), "The Last Man II" (2008), and "The Last Man III" (2013).

AUGMENTED PROMPT
Use only the context to answer. If unknown, say you don't know.

Context:
Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016.  Boria defeated former Miami-Dade School Board member Frank Bolaños in the 2012 elections, obtaining 54% of the vote

Biography 
Boria, who is also an evangelical Christian pastor, was born in Caracas in 1958 to Italian parents who emigrated to that country.  He studied accounting at the Andrés Bello Catholic University, a private institution, in 1982

In 1989 he left Venezuela to settle in Doral, Florida, where he currently lives.  Boria

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 7: In the RAG prompt construction step, we simply concatenate the retrieved chunks before the question. Discuss potential issues with this naive approach, such as token limits, redundancy, or irrelevant context dilution. Then, explain how we could select, weight, or summarize the retrieved chunks before injecting them into the prompt to improve generation quality and efficiency. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

In the naive RAG approach, simply concatenating all retrieved chunks before the question can lead to several issues. First, token limits of language models (for example, 4096 tokens for many models) may be exceeded, forcing truncation and potentially omitting critical information. Second, redundancy arises when chunks contain overlapping content, wasting tokens on duplicate data and reducing the effective context. Third, irrelevant context dilution occurs if less relevant chunks are included, introducing noise that can mislead the model and degrade answer quality by overshadowing key details.

To address these issues, we can enhance RAG by intelligently processing chunks before injection. Selection methods, such as choosing top-k chunks based on relevance scores or diversity criteria (for example, Maximal Marginal Relevance), prioritize the most useful information. Weighting approaches, like attention-based scoring or prompt engineering to emphasize high-relevance chunks, help the model focus on key content. Alternatively, summarization—using extractive methods to select salient sentences or abstractive techniques to generate concise summaries—reduces token usage while preserving essential context, improving both efficiency and generation accuracy.

#### **To go further**

- Experiment with other chunking methods (e.g., semantic or recursive chunking).  
- Explore **LangChain** and **LlamaIndex** for building modular RAG pipelines.  
- Try **hybrid retrieval** combining sparse (BM25) and dense embeddings.  
- Explore more advanced RAG methods such as **RAG-Fusion**, **Self-RAG**, and **Active-RAG**.  
- Experiment with **Matryoshka Representation Learning** for scalable embeddings.  
- Try **fine-tuning rerankers** or **retrievers** for domain-specific data.  
